# Comparison Between TreeValue and Tianshou Batch

In this section, we will take a look at the feature and performance of the [Tianshou Batch](https://github.com/thu-ml/tianshou) library, which is developed by Tsinghua Machine Learning Group.

Before starting the comparison, let us define some thing.

In [1]:
import torch 

_TREE_DATA_1 = {'a': 1, 'b': 2, 'x': {'c': 3, 'd': 4}}
_TREE_DATA_2 = {
    'a': torch.randn(2, 3), 
    'x': {
        'c': torch.randn(3, 4)
    },
}
_TREE_DATA_3 = {
    'obs': torch.randn(4, 84, 84),
    'action': torch.randint(0, 6, size=(1,)),
    'reward': torch.rand(1),
}

## Read and Write Operation

Reading and writing are the two most common operations in the tree data structure based on the data model (TreeValue and Tianshou Batch both belong to this type), so this section will compare the reading and writing performance of these two libraries.

### TreeValue's Get and Set

In [2]:
from treevalue import FastTreeValue

t = FastTreeValue(_TREE_DATA_2)

In [3]:
t

<FastTreeValue 0x7fd73874dfd0>
├── 'a' --> tensor([[ 2.0882,  0.6706, -0.4951],
│                   [ 0.8026,  1.4073, -0.6093]])
└── 'x' --> <FastTreeValue 0x7fd73b7fab50>
    └── 'c' --> tensor([[ 2.0969, -0.8406,  1.5021, -2.2032],
                        [-0.5722, -0.9777, -2.2600, -0.7326],
                        [ 1.7146, -0.5840,  0.4905, -1.8891]])

In [4]:
t.a

tensor([[ 2.0882,  0.6706, -0.4951],
        [ 0.8026,  1.4073, -0.6093]])

In [5]:
%timeit t.a

71.1 ns ± 0.0931 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [6]:
new_value = torch.randn(2, 3)
t.a = new_value

t

<FastTreeValue 0x7fd73874dfd0>
├── 'a' --> tensor([[-0.8485, -0.3824, -0.4923],
│                   [-2.1136, -1.0377,  0.1438]])
└── 'x' --> <FastTreeValue 0x7fd73b7fab50>
    └── 'c' --> tensor([[ 2.0969, -0.8406,  1.5021, -2.2032],
                        [-0.5722, -0.9777, -2.2600, -0.7326],
                        [ 1.7146, -0.5840,  0.4905, -1.8891]])

In [7]:
%timeit t.a = new_value

82.1 ns ± 0.168 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


### Tianshou Batch's Get and Set

In [8]:
from tianshou.data import Batch

b = Batch(**_TREE_DATA_2)

In [9]:
b

Batch(
    a: tensor([[ 2.0882,  0.6706, -0.4951],
               [ 0.8026,  1.4073, -0.6093]]),
    x: Batch(
           c: tensor([[ 2.0969, -0.8406,  1.5021, -2.2032],
                      [-0.5722, -0.9777, -2.2600, -0.7326],
                      [ 1.7146, -0.5840,  0.4905, -1.8891]]),
       ),
)

In [10]:
b.a

tensor([[ 2.0882,  0.6706, -0.4951],
        [ 0.8026,  1.4073, -0.6093]])

In [11]:
%timeit b.a

65.2 ns ± 0.178 ns per loop (mean ± std. dev. of 7 runs, 10,000,000 loops each)


In [12]:
new_value = torch.randn(2, 3)
b.a = new_value

b

Batch(
    a: tensor([[ 1.2011,  1.5019,  0.8877],
               [ 3.0177,  1.7704, -0.4007]]),
    x: Batch(
           c: tensor([[ 2.0969, -0.8406,  1.5021, -2.2032],
                      [-0.5722, -0.9777, -2.2600, -0.7326],
                      [ 1.7146, -0.5840,  0.4905, -1.8891]]),
       ),
)

In [13]:
%timeit b.a = new_value

540 ns ± 0.118 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


## Initialization

### TreeValue's Initialization

In [14]:
%timeit FastTreeValue(_TREE_DATA_1)

893 ns ± 2.21 ns per loop (mean ± std. dev. of 7 runs, 1,000,000 loops each)


### Tianshou Batch's Initialization

In [15]:
%timeit Batch(**_TREE_DATA_1)

11.1 µs ± 23.8 ns per loop (mean ± std. dev. of 7 runs, 100,000 loops each)


## Deep Copy Operation

In [16]:
import copy

### Deep Copy of TreeValue

In [17]:
t3 = FastTreeValue(_TREE_DATA_3)
%timeit copy.deepcopy(t3)

145 µs ± 1.08 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Deep Copy of Tianshou Batch

In [18]:
b3 = Batch(**_TREE_DATA_3)
%timeit copy.deepcopy(b3)

146 µs ± 999 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


## Stack, Concat and Split Operation

### Performance of TreeValue

In [19]:
trees = [FastTreeValue(_TREE_DATA_2) for _ in range(8)]

In [20]:
t_stack = FastTreeValue.func(subside=True)(torch.stack)

t_stack(trees)

<FastTreeValue 0x7fd7387842b0>
├── 'a' --> tensor([[[ 2.0882,  0.6706, -0.4951],
│                    [ 0.8026,  1.4073, -0.6093]],
│           
│                   [[ 2.0882,  0.6706, -0.4951],
│                    [ 0.8026,  1.4073, -0.6093]],
│           
│                   [[ 2.0882,  0.6706, -0.4951],
│                    [ 0.8026,  1.4073, -0.6093]],
│           
│                   [[ 2.0882,  0.6706, -0.4951],
│                    [ 0.8026,  1.4073, -0.6093]],
│           
│                   [[ 2.0882,  0.6706, -0.4951],
│                    [ 0.8026,  1.4073, -0.6093]],
│           
│                   [[ 2.0882,  0.6706, -0.4951],
│                    [ 0.8026,  1.4073, -0.6093]],
│           
│                   [[ 2.0882,  0.6706, -0.4951],
│                    [ 0.8026,  1.4073, -0.6093]],
│           
│                   [[ 2.0882,  0.6706, -0.4951],
│                    [ 0.8026,  1.4073, -0.6093]]])
└── 'x' --> <FastTreeValue 0x7fd7387783d0>
    └── 'c' --> tensor([[[

In [21]:
%timeit t_stack(trees)

33 µs ± 50.2 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [22]:
t_cat = FastTreeValue.func(subside=True)(torch.cat)

t_cat(trees)

<FastTreeValue 0x7fd73871a730>
├── 'a' --> tensor([[ 2.0882,  0.6706, -0.4951],
│                   [ 0.8026,  1.4073, -0.6093],
│                   [ 2.0882,  0.6706, -0.4951],
│                   [ 0.8026,  1.4073, -0.6093],
│                   [ 2.0882,  0.6706, -0.4951],
│                   [ 0.8026,  1.4073, -0.6093],
│                   [ 2.0882,  0.6706, -0.4951],
│                   [ 0.8026,  1.4073, -0.6093],
│                   [ 2.0882,  0.6706, -0.4951],
│                   [ 0.8026,  1.4073, -0.6093],
│                   [ 2.0882,  0.6706, -0.4951],
│                   [ 0.8026,  1.4073, -0.6093],
│                   [ 2.0882,  0.6706, -0.4951],
│                   [ 0.8026,  1.4073, -0.6093],
│                   [ 2.0882,  0.6706, -0.4951],
│                   [ 0.8026,  1.4073, -0.6093]])
└── 'x' --> <FastTreeValue 0x7fd73871a640>
    └── 'c' --> tensor([[ 2.0969, -0.8406,  1.5021, -2.2032],
                        [-0.5722, -0.9777, -2.2600, -0.7326],
                 

In [23]:
%timeit t_cat(trees)

30.8 µs ± 34.7 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [24]:
t_split = FastTreeValue.func(rise=True)(torch.split)
tree = FastTreeValue({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1),
})

%timeit t_split(tree, 1)

69.2 µs ± 91.1 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


### Performance of Tianshou Batch

In [25]:
batches = [Batch(**_TREE_DATA_2) for _ in range(8)]

Batch.stack(batches)

Batch(
    x: Batch(
           c: tensor([[[ 2.0969, -0.8406,  1.5021, -2.2032],
                       [-0.5722, -0.9777, -2.2600, -0.7326],
                       [ 1.7146, -0.5840,  0.4905, -1.8891]],
              
                      [[ 2.0969, -0.8406,  1.5021, -2.2032],
                       [-0.5722, -0.9777, -2.2600, -0.7326],
                       [ 1.7146, -0.5840,  0.4905, -1.8891]],
              
                      [[ 2.0969, -0.8406,  1.5021, -2.2032],
                       [-0.5722, -0.9777, -2.2600, -0.7326],
                       [ 1.7146, -0.5840,  0.4905, -1.8891]],
              
                      [[ 2.0969, -0.8406,  1.5021, -2.2032],
                       [-0.5722, -0.9777, -2.2600, -0.7326],
                       [ 1.7146, -0.5840,  0.4905, -1.8891]],
              
                      [[ 2.0969, -0.8406,  1.5021, -2.2032],
                       [-0.5722, -0.9777, -2.2600, -0.7326],
                       [ 1.7146, -0.5840,  0.4905, -1.8891]],

In [26]:
%timeit Batch.stack(batches)

83.9 µs ± 306 ns per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [27]:
Batch.cat(batches)

Batch(
    x: Batch(
           c: tensor([[ 2.0969, -0.8406,  1.5021, -2.2032],
                      [-0.5722, -0.9777, -2.2600, -0.7326],
                      [ 1.7146, -0.5840,  0.4905, -1.8891],
                      [ 2.0969, -0.8406,  1.5021, -2.2032],
                      [-0.5722, -0.9777, -2.2600, -0.7326],
                      [ 1.7146, -0.5840,  0.4905, -1.8891],
                      [ 2.0969, -0.8406,  1.5021, -2.2032],
                      [-0.5722, -0.9777, -2.2600, -0.7326],
                      [ 1.7146, -0.5840,  0.4905, -1.8891],
                      [ 2.0969, -0.8406,  1.5021, -2.2032],
                      [-0.5722, -0.9777, -2.2600, -0.7326],
                      [ 1.7146, -0.5840,  0.4905, -1.8891],
                      [ 2.0969, -0.8406,  1.5021, -2.2032],
                      [-0.5722, -0.9777, -2.2600, -0.7326],
                      [ 1.7146, -0.5840,  0.4905, -1.8891],
                      [ 2.0969, -0.8406,  1.5021, -2.2032],
                   

In [28]:
%timeit Batch.cat(batches)

156 µs ± 1.37 µs per loop (mean ± std. dev. of 7 runs, 10,000 loops each)


In [29]:
batch = Batch({
    'obs': torch.randn(8, 4, 84, 84),
    'action': torch.randint(0, 6, size=(8, 1,)),
    'reward': torch.rand(8, 1)}
)

%timeit list(Batch.split(batch, 1, shuffle=False, merge_last=True))

356 µs ± 6.28 µs per loop (mean ± std. dev. of 7 runs, 1,000 loops each)
